Resource Awesome dataset

Financial
Weather
Then biology predicting certain disease

Main components of project  
supervised learning: classification, SQL, flask, MVP and building outward

VOICE MUSIC: DID I SING IT RIGHT OR HOW WELL DID I SING (KARAOKE MACHINES GRADING YOUR PERFORMANCE HOW?)
Maybe me singing and grading it by an arbitrary algorithm?
How to have it learn from the actual song and compare with my singing?
Maybe classify if a note is correct or not.
How to deal with rhythm though?
Time the song and the transition of each note. 
Also how to deal with ad lib and spontaneous changes that work?
This involves delving into image processing?

MVP would be just something that finds if you hit the right note regardless of rhythm? Where is the machine learning in that though? Dont I just have to take the decibels in?
then could add rhythm and maybe adlib

might need to table this topic for later


TYPHOON/HURRICANE CLASSIFICATION
something to do with typhoons
a huge super typhoon hit hong kong(my home) while I was there before the bootcamp started
classify typhoon strength? what they do now is measure wind speed in order to classify or do they take satellite pictures in order to find wind speed? buoys are used?
brief research shows that satellite images are now being used to estimate wind speeds. Buoys are definitely used. An article as late as 2016 still says pilots fly out to collect wind speed data.
could probably get data from HK weather observatory? or do it for hurricanes in US?
This requires delving into analyzing images.
Maybe if a emergency response is needed? 
Trying to find the right complexity. 

# Financial Problem Statement 1
Crypto currency winners or losers. Should all be losers at this point but could be recovering now? But relative to other cryptos I guess.
![crypto](https://raw.githubusercontent.com/dynogravelso/mcnulty/master/crypto_ideas.JPG "crypto ideas")  
Foreign currency exchange data. Easier to understand? Any country data?  
Question 1: Funds Data and Stock Data. Making portfolio yourself vs. buying funds. There is probably more there than just which one is better right? Also isn't that more regression than classification?  
Sharpe ratio or just mean/std

# Weather Problem Statement 1
![typhoon](https://raw.githubusercontent.com/dynogravelso/mcnulty/master/typhoon_ideas.JPG "typhoon ideas")
Classify as Major/Minor/Not a big deal
Features to find:
Hurricane Data (Pressure, Eye diameter, Rain, Speed, Temp)
Environment Data (Sea level, Global CO2 levels, Ice Cap, Weather)
Data found: Monthly Average Temperature and Precipitation


In [2]:
from __future__ import print_function, division
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNetCV
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

In [30]:
# import pickle
# with open('all_storm_info' + '.pkl', 'wb') as picklefile:
#     pickle.dump(total_df, picklefile)

In [3]:
import pickle
total_df = pd.read_pickle('all_storm_info.pkl')

In [4]:
total_df[2]['EVENT_ID'] = total_df[2]['EVENT_ID'].apply(lambda x: int(x))

In [5]:
total_df[0].describe()

,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EPISODE_ID,EVENT_ID,STATE_FIPS,YEAR,CZ_FIPS,INJURIES_DIRECT,INJURIES_INDIRECT,DEATHS_DIRECT,DEATHS_INDIRECT,MAGNITUDE,CATEGORY,TOR_LENGTH,TOR_WIDTH,TOR_OTHER_CZ_FIPS,BEGIN_RANGE,END_RANGE,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON
count,1.514185e+06,1.514185e+06,1.514185e+06,1.514185e+06,1.514185e+06,1.514185e+06,1.281946e+06,1.514185e+06,1.514184e+06,1.514185e+06,1.514185e+06,1.514185e+06,1.514185e+06,1.514185e+06,1.514185e+06,892217.000000,266.000000,263265.000000,263265.000000,2010.000000,819526.000000,819149.000000,936087.000000,936087.000000,758597.000000,758598.000000
mean,2.003663e+05,1.499595e+01,1.326502e+03,2.003663e+05,1.633056e+01,1.486190e+03,6.579533e+05,3.709782e+06,3.164434e+01,2.003604e+03,9.355019e+01,9.960011e-02,8.441505e-03,1.173503e-02,1.528215e-03,25.120205,1.375940,0.865781,31.020770,104.719403,2.367046,2.303580,37.844888,-92.620687,37.904947,-92.747085
std,1.158240e+03,9.094419e+00,6.671332e+02,1.158240e+03,9.102727e+00,6.032086e+02,7.501498e+05,3.574978e+06,1.710111e+01,1.158302e+01,1.094920e+02,4.014830e+00,2.060941e+00,6.390916e-01,5.860460e-02,36.501246,0.960358,8.107986,124.059144,83.616096,6.190648,4.712186,4.898166,34.638207,4.922463,38.065567
min,1.950010e+05,1.000000e+00,0.000000e+00,1.950010e+05,1.000000e+00,0.000000e+00,1.000000e+00,3.000000e+00,1.000000e+00,1.950000e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,-14.400000,-815.100000,-14.456000,-815.100000
25%,1.999010e+05,7.000000e+00,9.000000e+02,1.999010e+05,8.000000e+00,1.200000e+03,5.616800e+04,4.000790e+05,1.900000e+01,1.999000e+03,2.500000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.880000,1.000000,0.000000,0.000000,47.000000,0.000000,0.000000,34.400000,-97.610000,34.570000,-97.500000
50%,2.006060e+05,1.500000e+01,1.517000e+03,2.006060e+05,1.600000e+01,1.615000e+03,1.130000e+05,5.149453e+06,3.000000e+01,2.006000e+03,6.500000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.750000,1.000000,0.000000,0.000000,93.000000,1.000000,1.000000,37.980000,-91.550000,38.050000,-91.080000
75%,2.012050e+05,2.300000e+01,1.820000e+03,2.012050e+05,2.400000e+01,1.910000e+03,1.167428e+06,5.577620e+06,4.500000e+01,2.012000e+03,1.170000e+02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,52.000000,1.000000,0.000000,10.000000,139.000000,3.000000,3.000000,41.350000,-83.680000,41.380000,-83.270000
max,2.018070e+05,3.100000e+01,2.359000e+03,2.018070e+05,3.100000e+01,2.359000e+03,2.414827e+06,1.035852e+07,9.900000e+01,2.018000e+03,8.760000e+02,1.700000e+03,2.400000e+03,6.380000e+02,2.000000e+01,22000.000000,5.000000,2315.000000,4576.000000,810.000000,3749.000000,925.000000,70.502900,171.400000,70.434200,171.400000


In [6]:
total_df[2].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1233592 entries, 0 to 37060
Data columns (total 11 columns):
YEARMONTH         1233592 non-null object
EPISODE_ID        1233592 non-null object
EVENT_ID          1233592 non-null int64
LOCATION_INDEX    1233592 non-null object
RANGE             805483 non-null float64
AZIMUTH           805483 non-null object
LOCATION          1035654 non-null object
LATITUDE          971922 non-null float64
LONGITUDE         971922 non-null float64
LAT2              761185 non-null float64
LON2              761185 non-null float64
dtypes: float64(5), int64(1), object(5)
memory usage: 112.9+ MB


In [8]:
# MAKING A COMPLETE COMBINED dataframe
full_df = pd.merge(total_df[0], total_df[1], how = 'outer', on='EVENT_ID')
full_df = pd.merge(full_df, total_df[2], how = 'outer', on='EVENT_ID')

In [44]:
# full_df.info(verbose = True, null_counts = True)
# full_df.describe()
# full_df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1782715 entries, 0 to 1782714
Data columns (total 71 columns):
BEGIN_YEARMONTH       1782692 non-null float64
BEGIN_DAY             1782692 non-null float64
BEGIN_TIME            1782692 non-null float64
END_YEARMONTH         1782692 non-null float64
END_DAY               1782692 non-null float64
END_TIME              1782692 non-null float64
EPISODE_ID_x          1550453 non-null float64
EVENT_ID              1782715 non-null int64
STATE                 1782691 non-null object
STATE_FIPS            1782691 non-null float64
YEAR                  1782692 non-null float64
MONTH_NAME            1782692 non-null object
EVENT_TYPE            1782692 non-null object
CZ_TYPE               1782692 non-null object
CZ_FIPS               1782692 non-null float64
CZ_NAME               1781135 non-null object
WFO                   1657122 non-null object
BEGIN_DATE_TIME       1782692 non-null object
CZ_TIMEZONE           1782692 non-null object
END_

In [9]:
# import pickle
# with open('complete_df' + '.pkl', 'wb') as picklefile:
#     pickle.dump(full_df, picklefile)

In [10]:
# full_df.to_csv('complete', encoding='utf-8')

KeyboardInterrupt: 

In [11]:
columns_needed = list(full_df.columns)
columns_needed = ['BEGIN_YEARMONTH',
 'BEGIN_DAY',
 'BEGIN_TIME',
 'EVENT_ID',
 'STATE',
 'YEAR',
 'MONTH_NAME',
 'EVENT_TYPE',
 'INJURIES_DIRECT',
 'INJURIES_INDIRECT',
 'DEATHS_DIRECT',
 'DEATHS_INDIRECT',
 'DAMAGE_PROPERTY',
 'DAMAGE_CROPS',
 'MAGNITUDE',
 'MAGNITUDE_TYPE',
 'BEGIN_LAT',
 'BEGIN_LON',
 'END_LAT',
 'END_LON',
 'FAT_YEARMONTH',
 'FAT_DAY',
 'FAT_TIME',
 'FATALITY_ID',
 'FATALITY_AGE',
 'FATALITY_SEX']

In [12]:
full_df.info(verbose=True, null_counts = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1782715 entries, 0 to 1782714
Data columns (total 71 columns):
BEGIN_YEARMONTH       1782692 non-null float64
BEGIN_DAY             1782692 non-null float64
BEGIN_TIME            1782692 non-null float64
END_YEARMONTH         1782692 non-null float64
END_DAY               1782692 non-null float64
END_TIME              1782692 non-null float64
EPISODE_ID_x          1550453 non-null float64
EVENT_ID              1782715 non-null int64
STATE                 1782691 non-null object
STATE_FIPS            1782691 non-null float64
YEAR                  1782692 non-null float64
MONTH_NAME            1782692 non-null object
EVENT_TYPE            1782692 non-null object
CZ_TYPE               1782692 non-null object
CZ_FIPS               1782692 non-null float64
CZ_NAME               1781135 non-null object
WFO                   1657122 non-null object
BEGIN_DATE_TIME       1782692 non-null object
CZ_TIMEZONE           1782692 non-null object
END_

In [13]:
clean1_df = full_df[columns_needed]
clean1_df.info(verbose=True, null_counts = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1782715 entries, 0 to 1782714
Data columns (total 26 columns):
BEGIN_YEARMONTH      1782692 non-null float64
BEGIN_DAY            1782692 non-null float64
BEGIN_TIME           1782692 non-null float64
EVENT_ID             1782715 non-null int64
STATE                1782691 non-null object
YEAR                 1782692 non-null float64
MONTH_NAME           1782692 non-null object
EVENT_TYPE           1782692 non-null object
INJURIES_DIRECT      1782692 non-null float64
INJURIES_INDIRECT    1782692 non-null float64
DEATHS_DIRECT        1782692 non-null float64
DEATHS_INDIRECT      1782692 non-null float64
DAMAGE_PROPERTY      1263109 non-null object
DAMAGE_CROPS         1143812 non-null object
MAGNITUDE            925281 non-null float64
MAGNITUDE_TYPE       370900 non-null object
BEGIN_LAT            1199904 non-null float64
BEGIN_LON            1199904 non-null float64
END_LAT              1022414 non-null float64
END_LON              10

In [16]:
event_types = list(full_df.EVENT_TYPE.unique())
event_types
event_types_wanted = [
 'Thunderstorm Wind',
 'THUNDERSTORM WINDS/FLOODING',
 'THUNDERSTORM WINDS/FLASH FLOOD',
 'THUNDERSTORM WINDS LIGHTNING',
 'THUNDERSTORM WIND/ TREES',
 'THUNDERSTORM WIND/ TREE',
 'THUNDERSTORM WINDS FUNNEL CLOU',
 'THUNDERSTORM WINDS/HEAVY RAIN',
 'THUNDERSTORM WINDS HEAVY RAIN',
 'THUNDERSTORM WINDS/ FLOOD',
 'High Wind',
 'Strong Wind',
 'Tropical Storm',
 'Hurricane (Typhoon)',
 'Marine High Wind',
 'Marine Thunderstorm Wind',
 'Heavy Wind',
 'Marine Strong Wind',
 'Tropical Depression',
 'Hurricane',
 'Marine Tropical Storm',
 'Marine Hurricane/Typhoon',
 'Marine Tropical Depression']

In [17]:
wind_df = clean1_df[clean1_df.EVENT_TYPE.isin(event_types_wanted)]

In [18]:
wind_df.head()

,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,EVENT_ID,STATE,YEAR,MONTH_NAME,EVENT_TYPE,INJURIES_DIRECT,INJURIES_INDIRECT,DEATHS_DIRECT,DEATHS_INDIRECT,DAMAGE_PROPERTY,DAMAGE_CROPS,MAGNITUDE,MAGNITUDE_TYPE,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,FAT_YEARMONTH,FAT_DAY,FAT_TIME,FATALITY_ID,FATALITY_AGE,FATALITY_SEX
1867,195506.0,10.0,1500.0,9976778,ALABAMA,1955.0,June,Thunderstorm Wind,0.0,0.0,0.0,0.0,0,0,0.0,NaN,33.00,-86.78,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1869,195507.0,28.0,1500.0,9976780,ALABAMA,1955.0,July,Thunderstorm Wind,0.0,0.0,0.0,0.0,0,0,52.0,NaN,33.58,-86.78,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1874,195510.0,28.0,1640.0,9976786,ALABAMA,1955.0,October,Thunderstorm Wind,0.0,0.0,0.0,0.0,0,0,0.0,NaN,33.58,-86.78,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1881,195503.0,3.0,2100.0,10005140,INDIANA,1955.0,March,Thunderstorm Wind,0.0,0.0,0.0,0.0,0,0,0.0,NaN,41.00,-85.18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1883,195505.0,15.0,1400.0,10000327,GEORGIA,1955.0,May,Thunderstorm Wind,0.0,0.0,0.0,0.0,0,0,0.0,NaN,33.00,-85.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
wind_df.info(verbose=True, null_counts = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 568137 entries, 1867 to 1782691
Data columns (total 26 columns):
BEGIN_YEARMONTH      568137 non-null float64
BEGIN_DAY            568137 non-null float64
BEGIN_TIME           568137 non-null float64
EVENT_ID             568137 non-null int64
STATE                568137 non-null object
YEAR                 568137 non-null float64
MONTH_NAME           568137 non-null object
EVENT_TYPE           568137 non-null object
INJURIES_DIRECT      568137 non-null float64
INJURIES_INDIRECT    568137 non-null float64
DEATHS_DIRECT        568137 non-null float64
DEATHS_INDIRECT      568137 non-null float64
DAMAGE_PROPERTY      428893 non-null object
DAMAGE_CROPS         354876 non-null object
MAGNITUDE            518682 non-null float64
MAGNITUDE_TYPE       370838 non-null object
BEGIN_LAT            447389 non-null float64
BEGIN_LON            447389 non-null float64
END_LAT              356463 non-null float64
END_LON              356464 non-null f

In [20]:
wind_df[wind_df.FATALITY_ID.isnull()].sort_values(by = 'DEATHS_DIRECT', ascending = False).head()

,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,EVENT_ID,STATE,YEAR,MONTH_NAME,EVENT_TYPE,INJURIES_DIRECT,INJURIES_INDIRECT,DEATHS_DIRECT,DEATHS_INDIRECT,DAMAGE_PROPERTY,DAMAGE_CROPS,MAGNITUDE,MAGNITUDE_TYPE,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,FAT_YEARMONTH,FAT_DAY,FAT_TIME,FATALITY_ID,FATALITY_AGE,FATALITY_SEX
540972,200203.0,9.0,1152.0,5287381,ILLINOIS,2002.0,March,High Wind,4.0,0.0,4.0,0.0,200K,NaN,50.0,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
587845,200307.0,10.0,1615.0,5334823,GEORGIA,2003.0,July,Thunderstorm Wind,0.0,0.0,3.0,0.0,35K,NaN,53.0,EG,33.77,-84.4,33.83,-84.37,NaN,NaN,NaN,NaN,NaN,NaN
683871,200409.0,4.0,2100.0,5421108,FLORIDA,2004.0,September,Tropical Storm,0.0,0.0,3.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
624929,200309.0,18.0,900.0,5370967,VIRGINIA,2003.0,September,Tropical Storm,0.0,0.0,2.0,0.0,9M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
631151,200311.0,13.0,500.0,5371814,PENNSYLVANIA,2003.0,November,High Wind,0.0,0.0,2.0,0.0,NaN,NaN,60.0,EG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
# import pickle
# with open('wind_df' + '.pkl', 'wb') as picklefile:
#     pickle.dump(wind_df, picklefile)

In [22]:
cn_pr_df = pd.read_excel('cn_pr_1991_2015.xls')
cn_tas_df = pd.read_excel('cn_tas_1991_2015.xls')
us_pr_df = pd.read_excel('us_pr_1901_2015.xls')
us_tas_df = pd.read_excel('us_tas_1901_2015.xls')
# us_tas_df.sort_values(by = '\tYear')

In [23]:
us_tas_df.columns

Index(['tas', '\tYear', ' Month', ' Country', ' ISO3', ' ISO2'], dtype='object')

In [24]:
# convert column into datetime
date_list = []
for i in range(0,1380):
    us_tas_df.iloc[i,4] = datetime(us_tas_df.iloc[i,1], us_tas_df.iloc[i,2],1).date()
us_tas_df.rename(columns = {' ISO3': 'BEGIN_YEARMONTH'}, inplace = True)
us_tas_df.drop(columns = ['\tYear', ' Month', ' ISO2'], inplace = True)

date_list = []
for i in range(0,1380):
    us_pr_df.iloc[i,4] = datetime(us_pr_df.iloc[i,1], us_pr_df.iloc[i,2],1).date() 
us_pr_df.rename(columns = {' ISO3': 'BEGIN_YEARMONTH'}, inplace = True)
us_pr_df.drop(columns = ['\tYear', ' Month', ' ISO2'], inplace = True)

# us_tas_df['BEGIN_YEARMONTH'] = datetime(us_tas_df['\tYear'], us_tas_df[' Month']) 

In [25]:
us_df = pd.merge(us_tas_df, us_pr_df, how = 'inner', on='BEGIN_YEARMONTH')
us_df.drop(columns = [' Country_y',' Country_x'], inplace = True)
climate_df = us_df
climate_df.head()

,tas,BEGIN_YEARMONTH,pr
0,-5.41530,1901-01-01,39.7591
1,-6.34610,1901-02-01,41.2308
2,-0.14910,1901-03-01,47.1593
3,5.00631,1901-04-01,49.7707
4,12.18910,1901-05-01,53.2810


In [46]:
# import pickle
# with open('climate_df' + '.pkl', 'wb') as picklefile:
#     pickle.dump(climate_df, picklefile)